In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import torch 
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
df_iris = pd.read_csv("./data/Iris.csv").drop(labels=["Id"],axis=1)

In [3]:
df_iris.head(3)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa


In [4]:
le = LabelEncoder()

y_encoded = le.fit_transform(df_iris["Species"])

In [5]:
x = torch.tensor(np.asarray(df_iris.iloc[:,0:-1]))

In [6]:
x.shape

torch.Size([150, 4])

In [7]:
x_train , x_test , y_train, y_test = train_test_split(x,y_encoded,test_size=.3,shuffle=True)

In [8]:
zero_count = 0
one_count  = 0
two_count  = 0

In [9]:
for i in y_train:
    
    if i == 0:
        zero_count+=1
    elif i == 1:
        one_count+=1
    else:
        two_count+=1

In [10]:
zero_count

36

In [11]:
one_count

38

In [12]:
two_count

31

In [13]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self,x,y):
        self.x = x
        self.y = y

    def __len__(self):

        return len(self.x)
    
    def __getitem__(self,idx):

        return self.x[idx],self.y[idx]

In [14]:
train_dataset = Dataset(x_train,y_train)
test_dataset  = Dataset(x_test,y_test)

In [15]:
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=16,shuffle=True)
test_loader  = torch.utils.data.DataLoader(test_dataset,batch_size=16,shuffle = True)

In [16]:
class FFN (torch.nn.Module):

    def __init__(self,input_size):
        super(FFN,self).__init__()
        self.relu    = torch.nn.ReLU()
        self.sigmoid = torch.nn.Sigmoid()

        self.layer1 = torch.nn.Linear(input_size,20)
        self.layer2 = torch.nn.Linear(20,10)
        self.layer3 = torch.nn.Linear(10,3)


    def forward(self,input):
      
        out = self.relu(self.layer1(input))
        out = self.relu(self.layer2(out))
        out = self.relu(self.layer3(out))

        return out

    

In [17]:
model = FFN(4)

In [20]:
model.train()

epochs = 12
optim  = torch.optim.AdamW(model.parameters(),lr=.0001)
loss_fn = torch.nn.CrossEntropyLoss()


for epoch in range(epochs):
    loss_per_epoch = 0
    for batch in train_loader:

        optim.zero_grad()

        x,y  = batch
      
        y_hat = model(x.float())

        loss = loss_fn(y_hat,y)

        loss.backward()

        optim.step()

        loss_per_epoch = loss.item()

    print(f"Epoch: {epoch}  Loss: {loss_per_epoch}")

Epoch: 0  Loss: 0.917884111404419
Epoch: 1  Loss: 0.6639776229858398
Epoch: 2  Loss: 0.7390135526657104
Epoch: 3  Loss: 0.6966472268104553
Epoch: 4  Loss: 0.5925883650779724
Epoch: 5  Loss: 0.5874601006507874
Epoch: 6  Loss: 0.6184017062187195
Epoch: 7  Loss: 0.7604634761810303
Epoch: 8  Loss: 0.812886118888855
Epoch: 9  Loss: 0.8905894756317139
Epoch: 10  Loss: 0.7575196623802185
Epoch: 11  Loss: 0.7327530980110168
